In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
from scipy.io import wavfile as wav
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

In [ ]:
# Display directory contents
!dir

EDA

In [ ]:
# Load and display audio waveforms for the first file
filename='UrbanSound8K/dog_bark.wav'
plt.figure(figsize=(14,5))
data,sample_rate = librosa.load(filename)
librosa.display.waveplot(data, sr=sample_rate)
ipd.Audio(filename)

In [ ]:
# Load and display audio waveforms for the second file
filename='UrbanSound8K/100263-2-0-3.wav'
plt.figure(figsize=(14,5))
data,sample_rate = librosa.load(filename)
librosa.display.waveplot(data, sr=sample_rate)
ipd.Audio(filename)

In [ ]:
# Sample rate details
sample_rate
wave_sample_rate, wave_audio = wav.read(filename)
wave_sample_rate

In [ ]:
# Display audio data details
wave_audio
data

In [ ]:
# Read metadata
metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
metadata.head(10)

In [ ]:
# Check whether the dataset is imbalanced
metadata['class'].value_counts()

#### Audio Classification Data Preprocessing

In [ ]:
# Read and plot a sample audio file using librosa
audio_file_path = 'UrbanSound8K/100263-2-0-3.wav'
librosa_audio_data, librosa_sample_rate = librosa.load(audio_file_path)

plt.figure(figsize=(12, 4))
plt.plot(librosa_audio_data)

In [ ]:
# Read and plot audio with scipy
wave_sample_rate, wave_audio = wav.read(audio_file_path)

plt.figure(figsize=(12, 4))
plt.plot(wave_audio)

#### Observation
Here Librosa converts the signal to mono, meaning the channel will alays be 1

### Extract Features
Here we will be using Mel-Frequency Cepstral Coefficients(MFCC) from the audio
samples.
The MFCC summarises the frequency distribution across the window size, so it is possible to analyse both the frequency and time characteristics of the sound. These audio representations will allow us to identify features for classification.

In [ ]:
mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)
print(mfccs.shape)

(40, 173)


In [ ]:
mfccs

array([[-4.45197296e+02, -4.47219299e+02, -4.49755127e+02, ...,
        -4.77412781e+02, -4.74241730e+02, -4.82704987e+02],
       [ 1.12513969e+02,  1.11970383e+02,  1.12244164e+02, ...,
         1.12045395e+02,  1.12248581e+02,  1.05560913e+02],
       [-1.58260956e+01, -2.30021858e+01, -3.12500191e+01, ...,
        -9.15441895e+00, -1.03232269e+01, -7.39410734e+00],
       ...,
       [-7.82766485e+00, -5.03880405e+00, -4.48165369e+00, ...,
        -1.90692782e-01,  4.34143972e+00,  1.00339069e+01],
       [-1.91763425e+00, -8.02737713e-01, -1.20930457e+00, ...,
        -1.23640239e-01,  2.90504694e-02,  9.22017097e-01],
       [-3.88130605e-01,  3.09317827e-01,  6.72155714e+00, ...,
        -2.33736587e+00, -4.25179911e+00, -2.31322765e+00]], dtype=float32)

In [ ]:
#### Extracting MFCC's For every audio file
import pandas as pd
import os
import librosa

audio_dataset_path='UrbanSound8K/audio/'
metadata=pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    return mfccs_scaled_features


In [ ]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

3554it [02:08, 27.26it/s]C:\Users\win10\anaconda3\envs\tensorflow\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1323
  warnings.warn(
8323it [04:54, 38.45it/s]C:\Users\win10\anaconda3\envs\tensorflow\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1103
  warnings.warn(
C:\Users\win10\anaconda3\envs\tensorflow\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1523
  warnings.warn(
8732it [05:08, 28.33it/s]


In [ ]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-215.79301, 71.66612, -131.81377, -52.091335,...",dog_bark
1,"[-424.68677, 110.56227, -54.148235, 62.01074, ...",children_playing
2,"[-459.56467, 122.800354, -47.92471, 53.265697,...",children_playing
3,"[-414.55377, 102.896904, -36.66495, 54.18041, ...",children_playing
4,"[-447.397, 115.0954, -53.809113, 61.60859, 1.6...",children_playing


In [ ]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
X.shape

(8732, 40)

In [ ]:
y

array(['dog_bark', 'children_playing', 'children_playing', ...,
       'car_horn', 'car_horn', 'car_horn'], dtype='<U16')

In [ ]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
X_train

array([[[-1.3183614e+02,  1.1397464e+02, -2.3956861e+01, ...,
          3.3314774e+00, -1.4786109e+00,  2.8736601e+00]],

       [[-1.4074220e+01,  9.1916939e+01, -8.6787214e+00, ...,
         -3.3844023e+00, -5.2119045e+00, -1.5936136e+00]],

       [[-4.9532028e+01,  1.5521857e-01, -2.0369110e+01, ...,
          2.0491767e+00, -8.0537486e-01,  2.7793026e+00]],

       ...,

       [[-4.2699332e+02,  9.2890656e+01,  3.0233388e+00, ...,
          8.6335975e-01,  6.4766806e-01,  7.8490508e-01]],

       [[-1.4607024e+02,  1.3709459e+02, -3.4298344e+01, ...,
          1.3777871e+00, -1.9530845e+00, -8.9652127e-01]],

       [[-4.2167450e+02,  2.1169032e+02,  2.6820304e+00, ...,
         -5.1484952e+00, -3.6400862e+00, -1.3321606e+00]]], dtype=float32)

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
X_train.shape

(6985, 1, 40)

In [ ]:
X_test.shape

(1747, 1, 40)

In [ ]:
y_train.shape

(6985, 10)

In [ ]:
y_test.shape

(1747, 10)

### Model Creation

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.1


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
### No of classes
num_labels=y.shape[1]

In [ ]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               4100      
_________________________________________________________________
activation_8 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_9 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)              

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
195/219 [=========================>....] - ETA: 0s - loss: 0.8612 - accuracy: 0.7104
Epoch 00001: val_loss improved from inf to 0.65317, saving model to saved_models\audio_classification.hdf5
219/219 [==============================] - 1s 3ms/step - loss: 0.8681 - accuracy: 0.7111 - val_loss: 0.6532 - val_accuracy: 0.7985
Epoch 2/100
204/219 [==========================>...] - ETA: 0s - loss: 0.8822 - accuracy: 0.7145
Epoch 00002: val_loss did not improve from 0.65317
219/219 [==============================] - 1s 2ms/step - loss: 0.8762 - accuracy: 0.7151 - val_loss: 0.6665 - val_accuracy: 0.7951
Epoch 3/100
200/219 [==========================>...] - ETA: 0s - loss: 0.8449 - accuracy: 0.7205
Epoch 00003: val_loss did not improve from 0.65317
219/219 [==============================] - 1s 2ms/step - loss: 0.8508 - accuracy: 0.7187 - val_loss: 0.6554 - val_accuracy: 0.7939
Epoch 4/100
217/219 [============================>.] - ETA: 0s - loss: 0.8584 - accuracy: 0.7154
Epoch 0000

Epoch 30/100
195/219 [=========================>....] - ETA: 0s - loss: 0.8480 - accuracy: 0.7220
Epoch 00030: val_loss did not improve from 0.63746
219/219 [==============================] - 0s 2ms/step - loss: 0.8494 - accuracy: 0.7205 - val_loss: 0.6511 - val_accuracy: 0.7985
Epoch 31/100
218/219 [============================>.] - ETA: 0s - loss: 0.8415 - accuracy: 0.7241
Epoch 00031: val_loss did not improve from 0.63746
219/219 [==============================] - 0s 2ms/step - loss: 0.8423 - accuracy: 0.7238 - val_loss: 0.6773 - val_accuracy: 0.7865
Epoch 32/100
198/219 [==========================>...] - ETA: 0s - loss: 0.8698 - accuracy: 0.7159
Epoch 00032: val_loss did not improve from 0.63746
219/219 [==============================] - 0s 2ms/step - loss: 0.8672 - accuracy: 0.7157 - val_loss: 0.6709 - val_accuracy: 0.8014
Epoch 33/100
209/219 [===========================>..] - ETA: 0s - loss: 0.8221 - accuracy: 0.7267
Epoch 00033: val_loss did not improve from 0.63746
219/219 [==

202/219 [==========================>...] - ETA: 0s - loss: 0.8256 - accuracy: 0.7200
Epoch 00059: val_loss did not improve from 0.63746
219/219 [==============================] - 0s 2ms/step - loss: 0.8240 - accuracy: 0.7215 - val_loss: 0.6396 - val_accuracy: 0.7979
Epoch 60/100
202/219 [==========================>...] - ETA: 0s - loss: 0.8444 - accuracy: 0.7186
Epoch 00060: val_loss did not improve from 0.63746
219/219 [==============================] - 0s 2ms/step - loss: 0.8435 - accuracy: 0.7185 - val_loss: 0.6671 - val_accuracy: 0.7853
Epoch 61/100
198/219 [==========================>...] - ETA: 0s - loss: 0.8411 - accuracy: 0.7208
Epoch 00061: val_loss did not improve from 0.63746
219/219 [==============================] - 0s 2ms/step - loss: 0.8385 - accuracy: 0.7214 - val_loss: 0.6507 - val_accuracy: 0.7956
Epoch 62/100
201/219 [==========================>...] - ETA: 0s - loss: 0.8107 - accuracy: 0.7253
Epoch 00062: val_loss did not improve from 0.63746
219/219 [===============

206/219 [===========================>..] - ETA: 0s - loss: 0.8426 - accuracy: 0.7221
Epoch 00088: val_loss did not improve from 0.63103
219/219 [==============================] - 0s 2ms/step - loss: 0.8396 - accuracy: 0.7231 - val_loss: 0.6475 - val_accuracy: 0.7939
Epoch 89/100
201/219 [==========================>...] - ETA: 0s - loss: 0.8247 - accuracy: 0.7284
Epoch 00089: val_loss did not improve from 0.63103
219/219 [==============================] - 0s 2ms/step - loss: 0.8270 - accuracy: 0.7248 - val_loss: 0.6569 - val_accuracy: 0.7934
Epoch 90/100
202/219 [==========================>...] - ETA: 0s - loss: 0.8377 - accuracy: 0.7232
Epoch 00090: val_loss did not improve from 0.63103
219/219 [==============================] - 0s 2ms/step - loss: 0.8452 - accuracy: 0.7213 - val_loss: 0.6453 - val_accuracy: 0.7985
Epoch 91/100
203/219 [==========================>...] - ETA: 0s - loss: 0.8140 - accuracy: 0.7306
Epoch 00091: val_loss did not improve from 0.63103
219/219 [===============

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8002289533615112


In [ ]:
prediction_feature.shape

(1, 40)

In [ ]:
X_test[1]

array([-466.1843    ,    1.5388278 ,  -34.397358  ,   35.715336  ,
        -15.166929  ,  -18.850813  ,   -0.7415805 ,  -15.99989   ,
        -21.354332  ,    7.6506834 ,  -29.031452  ,  -19.142824  ,
         -2.6798913 ,   -8.466884  ,  -14.7660475 ,   -7.004778  ,
         -7.103754  ,    8.887754  ,   14.911873  ,   21.47102   ,
         21.336624  ,    0.9169518 ,  -18.795404  ,   -5.001721  ,
         -0.70152664,    2.91399   ,   -6.7105994 ,  -16.638536  ,
         -9.821647  ,   12.8619585 ,    0.6552978 ,  -23.953394  ,
        -15.200551  ,    9.21079   ,   10.419799  ,   -0.57916117,
         -1.2440346 ,   17.722294  ,   13.837573  ,   -5.164349  ],
      dtype=float32)

In [ ]:
model.predict_classes(X_test)

array([5, 3, 4, ..., 1, 2, 2], dtype=int64)

### Testing Some Test Audio Data

Steps
- Preprocess the new audio data
- predict the classes
- Invere transform your Predicted Label

In [ ]:
filename="UrbanSound8K/drilling_1.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast')
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=model.predict_classes(mfccs_scaled_features)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label)
prediction_class

[-146.34639      52.85859       6.0391283    46.972637      0.48288426
   31.756617     -6.395756     36.949165     -2.2981966     9.0149975
   -8.056831     24.668858    -14.41076       7.5845594    -3.089655
   17.423319    -10.068965      9.606158     -1.4731672     7.745292
   -1.9399884    -1.5998945     3.373213      1.6671567    -4.9514785
    4.8195934    -6.1473813     3.8730834   -10.502274      1.3417107
   -5.616546      4.815169     -6.152183      2.0756485    -1.8508396
   -0.45990178   -4.9980536     2.528911     -0.7446382    -6.4779253 ]
[[-146.34639      52.85859       6.0391283    46.972637      0.48288426
    31.756617     -6.395756     36.949165     -2.2981966     9.0149975
    -8.056831     24.668858    -14.41076       7.5845594    -3.089655
    17.423319    -10.068965      9.606158     -1.4731672     7.745292
    -1.9399884    -1.5998945     3.373213      1.6671567    -4.9514785
     4.8195934    -6.1473813     3.8730834   -10.502274      1.3417107
    -5.616546 

array(['drilling'], dtype='<U16')